# Intelligent Systems Assignment 1

## Masterball solver

**Name:** Alexander Gonzalez Castañeda

**ID:** 1032452063


### 1. Create a class to model the Masterball problem

A Masterball must be represented as an array of arrays with integer values representing the color of the tile in each position:

A solved masterball must look like this:

```python
[ [0, 1, 2, 3, 4, 5, 6, 7],
  [0, 1, 2, 3, 4, 5, 6, 7],
  [0, 1, 2, 3, 4, 5, 6, 7],
  [0, 1, 2, 3, 4, 5, 6, 7]
]
```

#### Variables modeling the actions

In [1]:
'''
This variables MUST not be changed.
They represent the movements of the masterball.
'''
R_0 = "Right 0"
R_1 = "Right 1"
R_2 = "Right 2"
R_3 = "Right 3"

V_0 = "Vertical 0"
V_1 = "Vertical 1"
V_2 = "Vertical 2"
V_3 = "Vertical 3"
V_4 = "Vertical 4"
V_5 = "Vertical 5"
V_6 = "Vertical 6"
V_7 = "Vertical 7"


`R_i` moves the `i`th row to the right. For instance, `R_2` applied to the solved state will produce:

```python
[ [0, 1, 2, 3, 4, 5, 6, 7],
  [0, 1, 2, 3, 4, 5, 6, 7],
  [7, 0, 1, 2, 3, 4, 5, 6],
  [0, 1, 2, 3, 4, 5, 6, 7]
]
```

`V_i` performs a clockwise vertical move starting with the `i`th column

`V_1` applied to the above state will produce:

```python
[ [0, 4, 3, 2, 1, 5, 6, 7],
  [0, 3, 2, 1, 0, 5, 6, 7],
  [7, 4, 3, 2, 1, 4, 5, 6],
  [0, 4, 3, 2, 1, 5, 6, 7]
]
```

#### The Masterball problem class

In [2]:
import search
import util
import copy
class MasterballProblem(search.SearchProblem):    
    
    def __init__(self, startState):
        '''
        Store the initial state in the problem representation and any useful
        data.
        Here are some examples of initial states:
        [[0, 1, 4, 5, 6, 2, 3, 7], [0, 1, 3, 4, 5, 6, 3, 7], [1, 2, 4, 5, 6, 2, 7, 0], [0, 1, 4, 5, 6, 2, 3, 7]]
        [[0, 7, 4, 5, 1, 6, 2, 3], [0, 7, 4, 5, 0, 5, 2, 3], [7, 6, 3, 4, 1, 6, 1, 2], [0, 7, 4, 5, 1, 6, 2, 3]]
        [[0, 1, 6, 4, 5, 2, 3, 7], [0, 2, 6, 5, 1, 3, 4, 7], [0, 2, 6, 5, 1, 3, 4, 7], [0, 5, 6, 4, 1, 2, 3, 7]]
        '''
        self.startState = startState.__str__()
        self.expanded = 0
    
    def reset(self):
        self.expanded=0
    
    def readState(self, string):
        state=[]
        lists=string.split('], [')
        for i in lists:
            a=i.replace('[', '')
            a=a.replace(']', '')
            a=a.replace(' ', '')
            items=a.split(',')
            temp=[]
            for j in items:
                temp.append(int(j))
            state.append(temp)
        return state
    
    
    def makeMove(self, state, action):
        newState = self.readState(state)
        act=action.split()
        index=int(act[1])
        if act[0]=='Right':
            newState[index]=newState[index][-1:]+newState[index][:-1]
        else:
            transp=map(list, zip(*newState))
            temp=[]
            for i in range(0,4):
                j=(index+i)%8
                temp.append(transp[j])
                temp[i].reverse()
            temp.reverse()
            for i in range(0,4):
                j=(index+i)%8
                transp[j]=temp[i]
            newState=map(list, zip(*transp))
        return newState.__str__()
            
    
    def isGoalState(self, state):
        '''
        Define when a given state is a goal state (A correctly colored masterball)
        '''
        s = self.readState(state)
        transp=map(list, zip(*s))
        for idx, latitude in enumerate(transp):
            for item in latitude:
                if idx != item: return False
        return True
    
    def getStartState(self):
        '''
        Implement a method that returns the start state according to the SearchProblem
        contract.
        '''
        return self.startState

    def getSuccessors(self, state):
        '''
        Implement a successor function: Given a state from the masterball
        return a list of the successors and their corresponding actions. 

        This method *must* return a list where each element is a tuple of 
        three elements with the state of the masterball in the first position,
        the action (according to the definition above) in the second position, 
        and the cost of the action in the last position. 

        Note that you should not modify the state.
        '''
        self.expanded += 1
        successors=[]
        for i in range(0, 4):
            action='Right '+str(i)
            successors.append( (self.makeMove(state, action), action, 8) )
        for i in range(0, 8):
            action='Vertical '+str(i)
            successors.append( (self.makeMove(state, action), action, 16) )
        return successors

### 2. Implement iterative deepening search

Follow the example code provided in class and implement iterative deepening search (IDS).

In [3]:
import numpy as np
bF = 0

def iterativeDeepeningSearch(problem):
    global bF
    bF=0
    bFs=[]
    lim=50
    for i in range(1, lim+1):
        visited = {}
        state = problem.getStartState()
        visited[state] = 'gray'
        frontier = util.Stack()
        frontier.push((state, []))
        while not frontier.isEmpty():
            u, actions = frontier.pop()
            if len(actions)>=i: continue
            count =0
            for v, action, cost in problem.getSuccessors(u):
                if not v in visited:
                    count+=1
                    if problem.isGoalState(v):
                        bF = np.median(bFs)
                        return  actions + [action]
                    visited[v] = 'gray'
                    frontier.push((v, actions + [action]))
            visited[u] = 'green'
            bFs.append(count)
    bF = np.median(bFs)
    return []

def aStarSearch(problem, heuristic):
    global bF
    bF=0
    bFs=[]
    lim=50
    for i in range(1, lim+1):
        visited = {}
        state = problem.getStartState()
        visited[state] = 0
        frontier = util.PriorityQueue()
        frontier.push((state, [], 0),0)
        while not frontier.isEmpty():
            item=frontier.pop()
            u, actions, c = item
            if len(actions)>=i: continue
            count =0
            for v, action, cost in problem.getSuccessors(u):
                if v in visited and c+cost>=visited[v]: continue
                count+=1
                if problem.isGoalState(v):
                    bF = np.median(bFs)
                    return  actions + [action]
                visited[v] = c+cost
                frontier.push((v, actions + [action], c+cost), c+cost+heuristic(problem.readState(v)))
            bFs.append(count)
    bF = np.median(bFs)
    return []



Evaluate it to see what is the maximum depth that it could explore in a reasonable time. Report the results. 

### 3. Implement different heuristics for the problem

Implement at least two admissible and consistent heuristics. Compare A* using the heuristics against IDS calculating the number of expanded nodes and the effective branching factor, in the same way as it is done in figure 3.29 of [Russell10].

In [10]:
def myHeuristic1(state):
    transp=map(list, zip(*state))
    count=0
    for idx, latitude in enumerate(transp):
        for item in latitude:
            if idx == item: count+=1
    return 32-count

def myHeuristic2(state):
    solved = 0
    for index in xrange(0,8):
        if state[0][index] == state[1][index] == state[2][index] == state[3][index]:
            solved += 1
    return 8 - solved

In [12]:
def solveMasterBall(problem, search_function):
    '''
    This function receives a Masterball problem instance and a 
    search_function (IDS or A*S) and must return a list of actions that solve the problem.
    '''
    return search_function



problem = MasterballProblem([ [0, 4, 3, 2, 1, 5, 6, 7],
                              [0, 3, 2, 1, 0, 5, 6, 7],
                              [7, 4, 3, 2, 1, 4, 5, 6],
                              [0, 4, 3, 2, 1, 5, 6, 7]])

print '-- IDS --'
print solveMasterBall(problem, iterativeDeepeningSearch(problem))
print 'Nodes expanded:', problem.expanded, '  Branching Factor: ', bF
problem.reset()
print '\n-- A*(h1) --'
print solveMasterBall(problem, aStarSearch(problem, myHeuristic1))
print 'Nodes expanded:', problem.expanded, '  Branching Factor: ', bF
problem.reset()
print '\n-- A*(h2) --'
print solveMasterBall(problem, aStarSearch(problem, myHeuristic2))
print 'Nodes expanded:', problem.expanded, '  Branching Factor: ', bF


-- IDS --
['Vertical 5', 'Right 1', 'Vertical 5', 'Vertical 1']
Nodes expanded: 450   Branching Factor:  10.0

-- A*(h1) --
['Vertical 5', 'Right 1', 'Vertical 1', 'Vertical 5']
Nodes expanded: 345   Branching Factor:  10.0

-- A*(h2) --
['Vertical 5', 'Right 1', 'Vertical 1', 'Vertical 5']
Nodes expanded: 402   Branching Factor:  10.0
